In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
from tqdm import tqdm

### 1차 전처리

In [17]:
df = pd.read_csv('data/경기도.csv') 

In [18]:
df.head()

,시도,시군구,code,title,content
0,경기도,화성시,41590,<b>경기도 여행</b> 화성 숙소 푸르미르호텔 조식 먹고 호캉스,"봄날씨라 주말에는 집에만 있기 아까울정도에요~멀리 가기는 부담스럽고, 이번 주에는 ..."
1,경기도,화성시,41590,제부도 갈만한곳 <b>경기도 여행</b> 화성 바다 제부도 물때 시간표,제부도 갈만한곳 경기도 여행 화성 바다 제부도 물때 시간표​모세의 기적이 열리는 섬...
2,경기도,화성시,41590,서해랑 제부도 케이블카 전곡항 <b>경기도 여행</b> 바닷가,주말 가족과 함께 집에서 멀지 않은 제부도 나들이 다녀왔어요. 시화호 방조제 건너 ...
3,경기도,화성시,41590,<b>경기도 여행</b> 푸르미르호텔 조식 제대로인 화성 숙소 (+가볼만한곳),안녕하세요! 홍끄입니다 :)​최근 남편과 여행을 못간 것 같아집 근처인 경기도 화성...
4,경기도,화성시,41590,<b>경기도 여행</b> 서해랑 제부도 케이블카 대부도 바다향기 테마파크...,모세의 기적이 일어나는 제부도를 가려면 물때시간표를 보고 바닷길이 열릴 때 갈 수...


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12614 entries, 0 to 12613
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시도       12614 non-null  object
 1   시군구      12614 non-null  object
 2   code     12614 non-null  int64 
 3   title    12614 non-null  object
 4   content  12614 non-null  object
dtypes: int64(1), object(4)
memory usage: 492.9+ KB


In [20]:
# 중복행 개수 확인
duplicate_count = df.duplicated().sum()
print("중복된 행의 개수:", duplicate_count)

중복된 행의 개수: 1


In [21]:
# 중복행 제거 및 인덱스 재정렬
df = df.drop_duplicates(keep='first').reset_index(drop=True)

In [22]:
# 중복행 개수 확인
duplicate_count = df.duplicated().sum()
print("중복된 행의 개수:", duplicate_count)

중복된 행의 개수: 0


In [23]:
df['full_content'] = df['title'] + df['content']

In [24]:
# 불필요한 문자 제거 및 변환 함수
def remove_unnecessary_characters(text):
    # HTML 태그 제거
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    # HTML 엔티티 제거
    text = re.sub('&[^\s]*?;', '', text)
    # 한글, 영어, 공백을 제외한 모든 문자 제거
    pattern = r'[^가-힣a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    # 영어를 소문자로 변환
    text = text.lower()
    # 특정 문자 제거
    pattern = re.compile(r'naver|corp|copyright|next|image|\n|m|km')
    text = re.sub(pattern, '', text)
    # 공백이 두 칸 이상일 경우 한 칸으로 변환
    text = re.sub(r'\s+', ' ', text)
    return text

In [25]:
# 형태소(명사, 동사, 형용사, 알파벳) 추출 함수
def tokenize_text(text):
    pos_tags = okt.pos(text, norm=True, stem=True)  
    tokens = []
    for token, tag in pos_tags:
        if tag in ['Noun', 'Verb', 'Adjective', 'Alpha']:  
            tokens.append(token)
    return tokens

In [184]:
# 불용어 목록 파일을 불러와서 리스트로 저장하는 함수
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    return stopwords

In [185]:
# 불용어 제거 함수
def remove_stopwords(tokens):
    stopwords = load_stopwords(stopwords_file) 
    tokens = [token for token in tokens if token not in stopwords]
    tokens = ' '.join(tokens)
    return tokens

In [28]:
# 각 함수의 실행 시간 측정 및 진행 상태 표시
tqdm.pandas()

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 불용어 목록 파일
stopwords_file = 'text/stopwords.txt'

# 함수 실행
df['token'] = df['full_content'].progress_apply(remove_unnecessary_characters)
df['token'] = df['token'].progress_apply(tokenize_text)
df['token'] = df['token'].progress_apply(remove_stopwords)

100%|████████████████████████████████████████████████████████████████████████████| 12613/12613 [04:55<00:00, 42.64it/s]


In [29]:
df.head()

,시도,시군구,code,title,content,full_content,token
0,경기도,화성시,41590,<b>경기도 여행</b> 화성 숙소 푸르미르호텔 조식 먹고 호캉스,"봄날씨라 주말에는 집에만 있기 아까울정도에요~멀리 가기는 부담스럽고, 이번 주에는 ...",<b>경기도 여행</b> 화성 숙소 푸르미르호텔 조식 먹고 호캉스봄날씨라 주말에는 ...,숙소 푸르다 미르 호텔 조식 먹다 캉스 봄날 주말 집 아깝다 멀리 가기 부담스럽다 ...
1,경기도,화성시,41590,제부도 갈만한곳 <b>경기도 여행</b> 화성 바다 제부도 물때 시간표,제부도 갈만한곳 경기도 여행 화성 바다 제부도 물때 시간표​모세의 기적이 열리는 섬...,제부도 갈만한곳 <b>경기도 여행</b> 화성 바다 제부도 물때 시간표제부도 갈만한...,제부도 갈다 바다 제부도 물때 시간표 제부도 갈다 바다 제부도 물때 시간표 모세 기...
2,경기도,화성시,41590,서해랑 제부도 케이블카 전곡항 <b>경기도 여행</b> 바닷가,주말 가족과 함께 집에서 멀지 않은 제부도 나들이 다녀왔어요. 시화호 방조제 건너 ...,서해랑 제부도 케이블카 전곡항 <b>경기도 여행</b> 바닷가주말 가족과 함께 집에...,서해 제부도 케이블카 전곡항 바닷가 주말 가족 집 멀다 제부도 나들이 다녀오다 시화...
3,경기도,화성시,41590,<b>경기도 여행</b> 푸르미르호텔 조식 제대로인 화성 숙소 (+가볼만한곳),안녕하세요! 홍끄입니다 :)​최근 남편과 여행을 못간 것 같아집 근처인 경기도 화성...,<b>경기도 여행</b> 푸르미르호텔 조식 제대로인 화성 숙소 (+가볼만한곳)안녕하...,푸르다 미르 호텔 조식 제대로 숙소 안녕하다 홍 최근 남편 아집 근처 떠나다 요호 ...
4,경기도,화성시,41590,<b>경기도 여행</b> 서해랑 제부도 케이블카 대부도 바다향기 테마파크...,모세의 기적이 일어나는 제부도를 가려면 물때시간표를 보고 바닷길이 열릴 때 갈 수...,<b>경기도 여행</b> 서해랑 제부도 케이블카 대부도 바다향기 테마파크... 모세...,서해 제부도 케이블카 대부도 바다 향기 테마 파크 모세 기적 일어나다 제부도 물때 ...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12613 entries, 0 to 12612
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   시도            12613 non-null  object
 1   시군구           12613 non-null  object
 2   code          12613 non-null  int64 
 3   title         12613 non-null  object
 4   content       12613 non-null  object
 5   full_content  12613 non-null  object
 6   token         12613 non-null  object
dtypes: int64(1), object(6)
memory usage: 689.9+ KB


In [31]:
df.to_csv('data/경기도_전처리.csv', encoding='utf-8-sig', index=False)

### 2차 전처리

In [416]:
df = pd.read_csv('data/충청북도_전처리.csv') 

In [417]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4141 entries, 0 to 4140
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   시도            4141 non-null   object 
 1   시군구           4141 non-null   object 
 2   code          4141 non-null   float64
 3   title         4141 non-null   object 
 4   content       4141 non-null   object 
 5   full_content  4141 non-null   object 
 6   token         4141 non-null   object 
dtypes: float64(1), object(6)
memory usage: 226.6+ KB


In [418]:
# 불필요한 데이터 제거
word_list = '지역감각|지역전략|윤석열 대통령|급매|매매|전세|임대|월세|부동산|아파트|오피스텔|사업단|경진대회|프로그램|사업|노선|산학협력|분양|흥신소|교육|국토종합계획|대회'
df = df[~df['title'].str.contains(word_list)]

In [419]:
# 인덱스 재정렬
df = df.reset_index(drop=True)

In [420]:
# 타입 변환
df['code'] = df['code'].astype('int64')

In [421]:
# 영어 제거 
df['token'] = df['token'].str.replace(r'[a-zA-Z]+', '', regex=True)

In [422]:
# 불용어 제거
stopwords_file = 'text/stopwords2.txt'
df['token'] = df['token'].str.split().apply(remove_stopwords)

In [423]:
df = df[['시도', '시군구', 'code', 'token']]

In [424]:
df.head()

,시도,시군구,code,token
0,충청북도,괴산군,43760,광 저수지 은행나무 길 국내 가을 광 저수지 은행나무 길 광 저수지 은행나무 절정 ...
1,충청북도,괴산군,43760,군별 팔경 목록 여행명소 명소 군별 경 목록 정리 상당산성 둘러싸이다 구름 무심천 ...
2,충청북도,괴산군,43760,이야기 결과 후기 기간 수목 박일 여정 중부 지역 특이 사항 은퇴 부부 차 역사 인...
3,충청북도,괴산군,43760,드리다 스탬프 투어 스탬프 가이드 들락날락 블로그 시작 드리다 스탬프 투어 많다 참...
4,충청북도,괴산군,43760,베스트 좋다 지역 추천 청 충북도 둘리 숲속 둘리 숲속 속리산 법주사 길 위치 말티...


In [425]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4023 entries, 0 to 4022
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      4023 non-null   object
 1   시군구     4023 non-null   object
 2   code    4023 non-null   int64 
 3   token   4023 non-null   object
dtypes: int64(1), object(3)
memory usage: 125.8+ KB


In [426]:
df.to_csv('data/충청북도_전처리_2.csv', encoding='utf-8-sig', index=False)